# Census datasets presence

*Goal:* demonstrate basic use of the `datasets_presence_matrix` array.

The presence matrix is a sparse array, indicating which features (var) were present in each dataset.  The array has dimensions [n_datasets, n_var], and is stored in the SOMA Measurement `varp` collection. The first dimension is indexed by the `soma_joinid` in the `census_datasets` dataframe. The second is indexed by the `soma_joinid` in the `var` dataframe of the measurement.

In [1]:
import numpy as np
from scipy import sparse
import cell_census

census = cell_census.open_soma()
# Grab the experiment containing human data, and the measurement therein with RNA
human = census["census_data"]["homo_sapiens"]
human_rna = human.ms["RNA"]

# The cell census-wide datasets
datasets_df = census["census_info"]["datasets"].read().concat().to_pandas()

datasets_df

,soma_joinid,collection_id,collection_name,collection_doi,dataset_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
0,0,43d4bb39-21af-4d05-b973-4c1fed7b916c,Transcriptional Programming of Normal and Infl...,10.1016/j.celrep.2018.09.006,f512b8b6-369d-4a85-a695-116e0806857f,Skin,f512b8b6-369d-4a85-a695-116e0806857f.h5ad,68036
1,1,03cdc7f4-bd08-49d0-a395-4487c0e5a168,Emphysema Cell Atlas,,1e5bd3b8-6a0e-4959-8d69-cafed30fe814,immune cells,1e5bd3b8-6a0e-4959-8d69-cafed30fe814.h5ad,35699
2,2,03cdc7f4-bd08-49d0-a395-4487c0e5a168,Emphysema Cell Atlas,,214bf9eb-93db-48c8-8e3c-9bb22fa3bc63,AT2 cells,214bf9eb-93db-48c8-8e3c-9bb22fa3bc63.h5ad,3662
3,3,03cdc7f4-bd08-49d0-a395-4487c0e5a168,Emphysema Cell Atlas,,4b6af54a-4a21-46e0-bc8d-673c0561a836,non-immune cells,4b6af54a-4a21-46e0-bc8d-673c0561a836.h5ad,18386
4,4,0434a9d4-85fd-4554-b8e3-cf6c582bb2fa,Acute COVID-19 cohort across a range of WHO ca...,10.1101/2020.11.20.20227355,fa8605cf-f27e-44af-ac2a-476bee4410d3,PBMCs,fa8605cf-f27e-44af-ac2a-476bee4410d3.h5ad,59506
...,...,...,...,...,...,...,...,...
473,473,93eebe82-d8c3-41bc-a906-63b5b5f24a9d,Single-cell proteo-genomic reference maps of t...,10.1038/s41590-021-01059-0,cd4c96bb-ad66-4e83-ba9e-a7df8790eb12,3 healthy young and 3 healthy old bone marrow ...,cd4c96bb-ad66-4e83-ba9e-a7df8790eb12.h5ad,49057
474,474,93eebe82-d8c3-41bc-a906-63b5b5f24a9d,Single-cell proteo-genomic reference maps of t...,10.1038/s41590-021-01059-0,c05fb583-eb2f-4e3a-8e74-f9bd6414e418,healthy young bone marrow donor,c05fb583-eb2f-4e3a-8e74-f9bd6414e418.h5ad,13165
475,475,f70ebd97-b3bc-44fe-849d-c18e08fe773d,A transcriptomic atlas of the mouse cerebellum...,10.1101/2020.03.04.976407,e0ed3c55-aff6-4bb7-b6ff-98a2d90b890c,A transcriptomic atlas of the mouse cerebellum,e0ed3c55-aff6-4bb7-b6ff-98a2d90b890c.h5ad,611034
476,476,17481d16-ee44-49e5-bcf0-28c0780d8c4a,Single-Cell Sequencing of Developing Human Gut...,10.1016/j.devcel.2020.11.010,8e47ed12-c658-4252-b126-381df8d52a3d,Paediatric Human Gut (4-14y),8e47ed12-c658-4252-b126-381df8d52a3d.h5ad,22502


For convenience, read the entire presence matrix (for Homo sapiens) into a SciPy array. There is a convience API providing this capability, returning the matrix in a scipy.sparse.array:

In [2]:
presence_matrix = cell_census.get_presence_matrix(census, organism="Homo sapiens", measurement_name="RNA")

presence_matrix

<478x60664 sparse matrix of type '<class 'numpy.uint8'>'
	with 12536233 stored elements in Compressed Sparse Row format>

We also need the `var` dataframe, which is read into a Pandas DataFrame for convenient manipulation:

In [3]:
var_df = human_rna.var.read().concat().to_pandas()

var_df

,soma_joinid,feature_id,feature_name,feature_length
0,0,ENSG00000238009,RP11-34P13.7,3726
1,1,ENSG00000279457,WASH9P,1397
2,2,ENSG00000228463,AP006222.1,8224
3,3,ENSG00000237094,RP4-669L17.4,6204
4,4,ENSG00000230021,RP11-206L10.17,5495
...,...,...,...,...
60659,60659,ENSG00000288699,RP11-182N22.10,654
60660,60660,ENSG00000288700,RP11-22E12.2,6888
60661,60661,ENSG00000288710,RP11-386G11.12,2968
60662,60662,ENSG00000288711,AP000326.5,1307


## Is a feature present in a dataset?

*Goal:* test if a given feature is present in a given dataset.

**Important:** the presence matrix is indexed by soma_joinid, and is *NOT* positionally indexed.  In other words:
* the first dimension of the presence matrix is the dataset's `soma_joinid`, as stored in the `census_datasets` dataframe.
* the second dimension of the presence matrix is the feature's `soma_joinid`, as stored in the `var` dataframe.

In [4]:
var_joinid = var_df.loc[var_df.feature_id == "ENSG00000286096"].soma_joinid
dataset_joinid = datasets_df.loc[datasets_df.dataset_id == "97a17473-e2b1-4f31-a544-44a60773e2dd"].soma_joinid
is_present = presence_matrix[dataset_joinid, var_joinid][0, 0]
print(f'Feature is {"present" if is_present else "not present"}.')

Feature is present.


## What datasets contain a feature?

*Goal:* look up all datasets that have a feature_id present.

In [5]:
# Grab the feature's soma_joinid from the var dataframe
var_joinid = var_df.loc[var_df.feature_id == "ENSG00000286096"].soma_joinid

# The presence matrix is indexed by the joinids of the dataset and var dataframes,
# so slice out the feature of interest by its joinid.
dataset_joinids = presence_matrix[:, var_joinid].tocoo().row

# From the datasets dataframe, slice out the datasets which have a joinid in the list
datasets_df.loc[datasets_df.soma_joinid.isin(dataset_joinids)]

,soma_joinid,collection_id,collection_name,collection_doi,dataset_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
17,17,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,a68b64d8-aee3-4947-81b7-36b8fe5a44d2,Tabula Sapiens - Stromal,a68b64d8-aee3-4947-81b7-36b8fe5a44d2.h5ad,82478
18,18,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,97a17473-e2b1-4f31-a544-44a60773e2dd,Tabula Sapiens - Epithelial,97a17473-e2b1-4f31-a544-44a60773e2dd.h5ad,104148
19,19,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,c5d88abe-f23a-45fa-a534-788985e93dad,Tabula Sapiens - Immune,c5d88abe-f23a-45fa-a534-788985e93dad.h5ad,264824
20,20,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,5a11f879-d1ef-458a-910c-9b0bdfca5ebf,Tabula Sapiens - Endothelial,5a11f879-d1ef-458a-910c-9b0bdfca5ebf.h5ad,31691
21,21,e5f58829-1a66-40b5-a624-9046778e74f5,Tabula Sapiens,10.1126/science.abl4896,53d208b0-2cfd-4366-9866-c3c6114081bc,Tabula Sapiens - All Cells,53d208b0-2cfd-4366-9866-c3c6114081bc.h5ad,483152
330,330,283d65eb-dd53-496d-adb7-7570c7caa443,Transcriptomic diversity of cell types across ...,10.1101/2022.10.12.511898,07b1d7c8-5c2e-42f7-9246-26f746cd6013,Dissection: Myelencephalon (medulla oblongata)...,07b1d7c8-5c2e-42f7-9246-26f746cd6013.h5ad,27210
343,343,283d65eb-dd53-496d-adb7-7570c7caa443,Transcriptomic diversity of cell types across ...,10.1101/2022.10.12.511898,7c1c3d47-3166-43e5-9a95-65ceb2d45f78,Dissection: Pons (Pn) - Pontine reticular form...,7c1c3d47-3166-43e5-9a95-65ceb2d45f78.h5ad,49512
344,344,283d65eb-dd53-496d-adb7-7570c7caa443,Transcriptomic diversity of cell types across ...,10.1101/2022.10.12.511898,9372df2d-13d6-4fac-980b-919a5b7eb483,Dissection: Midbrain (M) - Periaqueductal gray...,9372df2d-13d6-4fac-980b-919a5b7eb483.h5ad,33794
372,372,283d65eb-dd53-496d-adb7-7570c7caa443,Transcriptomic diversity of cell types across ...,10.1101/2022.10.12.511898,dd03ce70-3243-4c96-9561-330cc461e4d7,Dissection: Cerebral cortex (Cx) - Perirhinal ...,dd03ce70-3243-4c96-9561-330cc461e4d7.h5ad,23732
386,386,283d65eb-dd53-496d-adb7-7570c7caa443,Transcriptomic diversity of cell types across ...,10.1101/2022.10.12.511898,7a0a8891-9a22-4549-a55b-c2aca23c3a2a,Supercluster: Hippocampal CA1-3,7a0a8891-9a22-4549-a55b-c2aca23c3a2a.h5ad,74979


## What features are in a dataset?

*Goal:* lookup the features present in a given dataset.

This example also demonstrates the ability to do the query on multiple datasets.

In [6]:
# Slice the dataset(s) of interest, and get the joinid(s)
dataset_joinids = datasets_df.loc[datasets_df.collection_id == "17481d16-ee44-49e5-bcf0-28c0780d8c4a"].soma_joinid

# Slice the presence matrix by the first dimension, i.e., by dataset
var_joinids = presence_matrix[dataset_joinids, :].tocoo().col

# From the feature (var) dataframe, slice out features which have a joinid in the list.
var_df.loc[var_df.soma_joinid.isin(var_joinids)]

,soma_joinid,feature_id,feature_name,feature_length
0,0,ENSG00000238009,RP11-34P13.7,3726
1,1,ENSG00000279457,WASH9P,1397
2,2,ENSG00000228463,AP006222.1,8224
3,3,ENSG00000237094,RP4-669L17.4,6204
4,4,ENSG00000230021,RP11-206L10.17,5495
...,...,...,...,...
55430,55430,ENSG00000279058,AGAP14P,2060
58483,58483,ENSG00000239511,POM121L7P,2404
59007,59007,ENSG00000229015,RP11-265D19.6,405
59120,59120,ENSG00000224963,ECMXP,2167
